<h1> Trabalho Música Formalizada Parte-2 </h1>

    Aluno: Natanael Luciano de Matos

caso não tenha o scamp instalado, execute as linha de codigo abaixo no seu cmd 
pip install scamp
pip3 install --user scamp_extensions

a visualização da partitura é feita usando lilypond

Site do scamp: http://scamp.marcevanstein.com/index.html

In [2]:
from scamp import *
import random
import scipy.stats as st
import numpy as np
#se o senhor receber uma mensagem de aviso, basta importar as bibliotecas novamente.

In [537]:
def Markov_matrix(size, func = st.uniform.rvs):
    #gera a matriz de transição com tamanho size*size, e com valores distribuidos segundo func
    A = func(size = size**2)
    A = A.reshape(size,size)
    for i in range(size):
        A[i][:] = A[i][:]/sum(A[i][:]) #normalizando as probabilidades
    A = np.matrix(A)
    #print('Matriz de transição = {}'.format(A))
    return A

In [520]:
#instruments range in midi
#cello
pitchs_cello = list(range(36,49))

#harp
pitchs_harp = list(range(30,61))

#violin
pitchs_violin = list(range(48,70))


In [521]:
s = Session()     #check if it´s all ok
Harp = s.new_part('Harp')
for i in pitchs_harp[::10]:
    Harp.play_note(i,1,1)


Using preset Harp LP2 for Harp


In [522]:
def note_gen(A,pitchs,n_notas,old):
    #função geradora de notas. gera as notas conforme uma cadeia de markov dada pela matriz A
    #gera  a = n_notas notas  
    u = np.zeros(len(pitchs))
    if old[0] == -1:
        z = random.randint(0, len(pitchs) -1 )
        u[z] = 1
        nota = [z + pitchs[0]]
    else:
        u[(old[0]-pitchs[0])] = 1
        nota = [old[0]]

    u = u.reshape(1,len(pitchs))
    for i in range(1,n_notas+1):
        vec =  u*(A**i) #vetor das probabilidades
        nota = nota +  list(random.choices(pitchs,weights = vec.T))
        
    if old[0] != -1:
        return nota[1:]
        
    return nota
    
    


In [523]:
A=Markov_matrix(len(pitchs_cello))


In [524]:
x = note_gen(A,pitchs_cello,10,[-1])

In [525]:
Cello = s.new_part('Cello')
for i in x:
    Cello.play_note(i,1,1) #notas escolhidas seguindo uma cadeia de Markov dada pela Matriz A

Using preset Cello LP for Cello


Para evitar que a música fique lenta na hora de tocar as notas. Irei construir a orquestra antes.
Utilizando o Xenaks como inspiração, primeiro selecionamos um instrumento e depois construímos a sequência de notas que serão tocadas pelo instrumento em um dado momento da música. As notas tem duração aleatória.
A quantidade de notas segue uma distribuição exponencial, mas foi realizada uma adaptação para que pelo menos uma nota seja tocada.

In [526]:
#escolhendo instrumentos:
#os instrumentos seram escolhidos de forma aleatória, assim como a quantidade de notas que iram tocar naquele momento
def create_orc(iterations = 20):
    #retorna uma sequencia de vetores contendo o instrumento e as notas que seram tocadas durante o concerto
    A_cello = Markov_matrix(len(pitchs_cello))
    A_harp  = Markov_matrix(len(pitchs_harp))
    A_vio   = Markov_matrix(len(pitchs_violin))
    cello_old = [-1]
    harp_old = [-1]
    vio_old = [-1]


    orchestra = [] # seq de vetores, onde primeira entrada é o instrumento e o resto as notas
    instrumentos = [0,1,2]  #"cello","violin","Harp"
    for i in range(iterations):
        inst = random.choice(instrumentos)
        n_notas = min(10,max(1,int(st.expon.rvs(loc =1, scale = 7))))
        if inst == 0:
            notes = note_gen(A_cello,pitchs_cello,n_notas,cello_old)
            cello_old[0] = notes[-1]
        
        elif inst == 1:
            notes = note_gen(A_vio,pitchs_violin,n_notas,vio_old)
            vio_old[0] = notes[-1]
        
        else: 
            notes = note_gen(A_harp,pitchs_harp,n_notas,harp_old)
            harp_old[0] = notes[-1]
            

        orchestra.append([inst] + notes) 
    return orchestra


In [527]:
vio = s.new_part('violin')

Using preset Violin LP3 for violin


In [528]:
Orc = create_orc(3)

In [529]:


s.start_transcribing()
for i in Orc:
    if i[0] == 0:
        for j in i[1:]:
            Cello.play_note(j,1,random.choice([1,2,.5,.25]))    
        
    elif i[0] == 1:
        for j in i[1:]:
            vio.play_note(j,1,random.choice([1,2,.5,.25]))    

    else:
        for j in i[1:]:
            Harp.play_note(j,1,random.choice([1,2,.5,.25])) 
s.stop_transcribing().to_score(title = 'Markov Orch',composer = 'Natanael Luciano').show_xml()
print('Done')

Done


Até o momento o volume das notas está constante, vamos mudar isso criando dinamicas musicais

In [530]:
forte_piano = Envelope.from_levels_and_durations(
    [0.8,0.4,1], [0.2,0.8], curve_shapes = [0,3]    #entendi a ideia, mas não sei exatamente
    #como fuinciona 'teoricamente' essa variação fortepiano. Peguei a ideia nos vídeos que 
    #andei assistindo
)

diminuendo = Envelope.from_levels([0.8,0.3])

A função abaixo cria uma sequencia contendo as dinamicas de cada nota, essa sequencia é dada por uma cadeia de markov.

In [531]:
def din_markov(A,n_notas,old):
    size = len(A)
    u = np.zeros(size)
    if old[0] == -1:
        z = random.randint(0, size -1 )
        u[z] = 1
        if z == 0:
            dinam = [1]
        elif z ==1:
            dinam = [forte_piano]
        else:
            dinam = [diminuendo]
    else:
        if old[0] == 1:
            u[0] = 1
        elif old[0] == forte_piano:
            u[1] = 1
        else:
            u[2] = 1
        
        dinam = [old[0]]

    u = u.reshape(1,size)
    for i in range(1,n_notas+1):
        vec =  u*(A**i) #vetor das probabilidades
        dinam = list(dinam) +  list(random.choices([1,forte_piano,diminuendo],weights = vec.T))
    #print(dinam) 
    
    if old[0] != -1:
        return dinam[1:]
    return dinam
    

Implementando na orchestra as dinamicas criadas acima .

In [538]:
def create_orc_2(iterations = 20):
    A_cello = Markov_matrix(len(pitchs_cello))
    A_harp  = Markov_matrix(len(pitchs_harp))
    A_vio   = Markov_matrix(len(pitchs_violin))
    A_din = Markov_matrix(3)
    cello_old = [-1]
    harp_old = [-1]
    vio_old = [-1]
    din_old = [-1]

    orchestra = [] # seq de vetores, onde primeira entrada é o instrumento e o resto as notas
    din = [] #seq contendo as dinamicas das notas
    instrumentos = [0,1,2]  #"cello","violin","Harp"
    for i in range(iterations):
        inst = random.choice(instrumentos)
        n_notas = min(10,max(1,int(st.expon.rvs(loc =1, scale = 7))))
        if inst == 0:
            notes = note_gen(A_cello,pitchs_cello,n_notas,cello_old)
            cello_old[0] = notes[-1]
        
        elif inst == 1:
            notes = note_gen(A_vio,pitchs_violin,n_notas,vio_old)
            vio_old[0] = notes[-1]
        
        else: 
            notes = note_gen(A_harp,pitchs_harp,n_notas,harp_old)
            harp_old[0] = notes[-1]
            
        dinamics = din_markov(A_din,n_notas,din_old)
        din_old[0] = dinamics[-1]
        if len(notes) > len(dinamics):
            dinamics.append(1)
        #print([len(notes),len(dinamics)])
        din.append(dinamics)
        orchestra.append([inst] + notes)
    return [orchestra, din]


O código abaixo gera a orchestra e toca as notas da mesma. No final, retorna um PDF da partitura.

In [542]:
Orc, d = create_orc_2(15)
s= Session()

Cello = s.new_part('Cello')
vio = s.new_part('Violin')
Harp = s.new_part('Harp')

s.start_transcribing()
for w,i in enumerate(Orc):
    if i[0] == 0:
        for j,p in zip(i[1:],d[w]):
            
            Cello.play_note(j,p,random.choice([1,2,.5,.25]))    
        
    elif i[0] == 1:
        for j,p in zip(i[1:],d[w]):
            vio.play_note(j,p,random.choice([1,2,.5,.25]))    
            
    else:
        for j,p in zip(i[1:],d[w]):
            Harp.play_note(j,p,random.choice([1,2,.5,.25]), blocking = random.choice([True,False,False])) 
            
s.stop_transcribing().to_score(title = 'Markov Orch',composer = 'Natanael Luciano').show()
print('Done')


Using preset Cello LP for Cello
Using preset Violin LP3 for Violin
Using preset Harp LP2 for Harp
Done


Mude 's.stop_transcribing().to_score(title = 'Markov Orch',composer = 'Natanael Luciano').show()' para 's.stop_transcribing().to_score(title = 'Markov Orch',composer = 'Natanael Luciano').show_xml()' se quiser gerar um arquivo Music xml

